In [266]:
import pandas as pd
from io import StringIO
import dateutil.parser as dp
import airtable

In [267]:
fed = pd.read_csv("../sample_data/Federal.csv")
ab = pd.read_csv("../sample_data/Alberta.csv")
nu = pd.read_csv("../sample_data/Nunuvat.csv")
bc = pd.read_csv("../sample_data/British_Columbia.csv")
yk = pd.read_csv("../sample_data/Yukon.csv")

In [268]:
col_df = airtable.get_df()

In [269]:
col_df.set_index("organization", inplace=True)

In [270]:
del col_df["id"]

In [271]:
sample_data = pd.DataFrame(columns=col_df.keys())

## Federal

In [272]:
# from https://stackoverflow.com/a/4766400

tz_str = '''-12 Y
-11 X NUT SST
-10 W CKT HAST HST TAHT TKT
-9 V AKST GAMT GIT HADT HNY
-8 U AKDT CIST HAY HNP PST PT
-7 T HAP HNR MST PDT
-6 S CST EAST GALT HAR HNC MDT
-5 R CDT COT EASST ECT EST ET HAC HNE PET
-4 Q AST BOT CLT COST EDT FKT GYT HAE HNA PYT
-3 P ADT ART BRT CLST FKST GFT HAA PMST PYST SRT UYT WGT
-2 O BRST FNT PMDT UYST WGST
-1 N AZOT CVT EGT
0 Z EGST GMT UTC WET WT
1 A CET DFT WAT WEDT WEST
2 B CAT CEDT CEST EET SAST WAST
3 C EAT EEDT EEST IDT MSK
4 D AMT AZT GET GST KUYT MSD MUT RET SAMT SCT
5 E AMST AQTT AZST HMT MAWT MVT PKT TFT TJT TMT UZT YEKT
6 F ALMT BIOT BTT IOT KGT NOVT OMST YEKST
7 G CXT DAVT HOVT ICT KRAT NOVST OMSST THA WIB
8 H ACT AWST BDT BNT CAST HKT IRKT KRAST MYT PHT SGT ULAT WITA WST
9 I AWDT IRKST JST KST PWT TLT WDT WIT YAKT
10 K AEST ChST PGT VLAT YAKST YAPT
11 L AEDT LHDT MAGT NCT PONT SBT VLAST VUT
12 M ANAST ANAT FJT GILT MAGST MHT NZST PETST PETT TVT WFT
13 FJST NZDT
11.5 NFT
10.5 ACDT LHST
9.5 ACST
6.5 CCT MMT
5.75 NPT
5.5 SLT
4.5 AFT IRDT
3.5 IRST
-2.5 HAT NDT
-3.5 HNT NST NT
-4.5 HLV VET
-9.5 MART MIT'''

tzd = {}
for tz_descr in map(str.split, tz_str.split('\n')):
    tz_offset = int(float(tz_descr[0]) * 3600)
    for tz_code in tz_descr[1:]:
        tzd[tz_code] = tz_offset

In [273]:
def parse_fed_time(x):
    y = x.split(" ")
    clean_str = y[0] + " " + y[1] + " " + y[-1][1:-1]
    return dp.parse(clean_str, tzinfos=tzd)

In [274]:
sample_data_fed = pd.DataFrame(columns=col_df.columns)
for col in col_df.columns:
    fed_col = col_df[col]["federal"]
    if fed_col == fed_col:
        sample_data_fed[col] = fed.query("language == 'English'")[fed_col].copy()        

In [275]:
sample_data_fed["closing_date"] = sample_data_fed["closing_date"].map(parse_fed_time)
sample_data_fed["closing_date"] = sample_data_fed["closing_date"].map(lambda x: str(x))

In [276]:
# I need to parse province_territory_of_work into a list of 2 letter codes
# ref: https://www12.statcan.gc.ca/census-recensement/2011/ref/dict/table-tableau/table-tableau-8-eng.cfm
prov_terr_dict = {
    "Newfoundland and Labrador": "NL",
    "Prince Edward Island": "PE",
    "Nova Scotia": "NS",
    "New Brunswick": "NB",
    "Quebec": "QC",
    "Ontario": "ON",
    "Manitoba": "MB",
    "Saskatchewan": "SK",
    "Alberta": "AB",
    "British Columbia": "BC",
    "Yukon": "YK",
    "Northwest Territories": "NT",
    "Nunavut": "NU",
    "National Capital Region": "ON, QC",
    "Canada": "NL, PE, NS, NB, QC, ON, MB, SK, AB, BC, YK, NT, NU",
    # will have to think about the locations below and how they will be represented in filtering
    "Aboriginal Lands": "",
    "Foreign": "",
    "United States": "",
    "World": "",
    "Europe": "",
    "Mexico": "",
    "Unspecified": ""
}

def replace_with_acronyms(x):
    for k in prov_terr_dict:
        x = x.replace(k, prov_terr_dict[k])
    return [y.strip() for y in x.split(",") if y]


In [277]:
sample_data_fed["province_territory_of_work"] = sample_data_fed["province_territory_of_work"].map(replace_with_acronyms)

In [278]:
def is_capital(letter):
    return letter.lower() != letter

def get_gsins(field):
    gsins = []
    for x in field.split(","):
        parts = x.strip().split(" - ")
        if len(parts) < 2:
            continue
        maybe_code = parts[0]
        if len(maybe_code) < 3:
            continue
        # 2nd char of code must be a capital letter or number, 3rd char a digit
        char_1 = maybe_code[1]
        char_2 = maybe_code[2]
        if is_capital(char_1) or char_1.isdigit():
            gsins.append(parts[0])
    return gsins

In [279]:
sample_data_fed["GSIN_code"] = sample_data_fed["GSIN_code"].map(get_gsins)

In [280]:
sample_data = pd.concat([sample_data, sample_data_fed], sort=False)

## BC

In [281]:
sample_data_bc = pd.DataFrame(columns=col_df.columns)
for col in col_df.columns:
    bc_col = col_df[col]["bc"].strip()
    if "," in bc_col:
        # we need to map more than 1 column, so skip for now
        continue
    if bc_col == bc_col:
        sample_data_bc[col] = bc[bc_col].copy()        

In [282]:
sample_data_bc["province_territory_of_work"] = sample_data_bc["province_territory_of_work"].map(lambda x: [x])

In [283]:
sample_data_bc["buyer_name"] = sample_data_bc["buyer_name"].map(lambda x: x.split("\n")[2])

In [284]:
sample_data_bc["published_date"] = sample_data_bc["published_date"].map(lambda x: pd.to_datetime(str(x)).date())
bc["CLOSING_DATETIME"] = bc["CLOSING_DATE"].map(str) + " " + bc["CLOSING_TIME"].map(lambda x: x.split(" ")[0]) + " " + bc["CLOSING_TIME"].map(lambda x: x.split(" ")[1]).map({"Local": "PST", "Pacific": "PST", "Mountain": "MST"})
sample_data_bc["closing_date"] = bc["CLOSING_DATETIME"].map(lambda x: str(dp.parse(x, tzinfos=tzd)))

In [285]:
sample_data = pd.concat([sample_data, sample_data_bc], sort=False)

## Nunavut

In [326]:
col_df

,GSIN_code,URL,buyer_contact,buyer_name,closing_date,location_of_work,province_territory_of_work,published_date,tender_description,title
organization,,,,,,,,,,
ab,Gsins,OriginSystemUrl,"ContactFirstName, ContactLastName, ContactPhon...",NaN,CloseDateUtc,RegionOfDelivery,Jurisdiction,PostDateUtc,"LongDescription, ShortDescription",Title
bc,GSIN,URL,"E_DETAIL_2, TELEPHONE",E_DETAIL_2,"CLOSING_DATE,CLOSING_TIME",REGION_DELIVERY,JURISDICTION,POSTING_DATE,E_DETAIL_1,E_TITLE
federal,gsin,NaN,contact,end_user_entity,date_closing,region_delivery,region_delivery,publication_date,description,title
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nu,NaN,NaN,"Contact Person, Phone Number, Email",NaN,ClosingDate,FOB Point Or Location,FOB Point Or Location,IssuedDate,NaN,Description


In [294]:
sample_data_nu = pd.DataFrame(columns=col_df.columns)
for col in col_df.columns:
    if pd.isna(col_df[col]["nu"]):
        continue
    nu_col = col_df[col]["nu"].strip()
    if "," in nu_col:
        # we need to map more than 1 column, so skip for now
        continue
    if nu_col == nu_col:
        sample_data_nu[col] = nu[nu_col].copy()        

In [323]:
def get_nu_location(_location):
    location_dict = {"Nunavut": "NU"}
    parts = _location.split(",")
    if len(parts) == 1:
        return location_dict[parts[0]]
    
    return parts[-1].strip()

In [325]:
sample_data_nu["province_territory_of_work"] = sample_data_nu["province_territory_of_work"].map(get_nu_location)

In [329]:
abc1 = sample_data_nu.closing_date[0]
abc2 = sample_data_nu.closing_date[2]

In [341]:
sample_data_nu["closing_date"] = sample_data_nu["closing_date"].map(lambda x: x if "EDT" in x else x + " 16:00 EDT").map(lambda x: str(dp.parse(x, tzinfos=tzd)))

In [342]:
sample_data_nu

,GSIN_code,URL,buyer_contact,buyer_name,closing_date,location_of_work,province_territory_of_work,published_date,tender_description,title
0,NaN,NaN,NaN,NaN,2019-07-02 16:00:00-04:00,"Iqaluit, NU",NU,2019-05-24,NaN,Custom Molded Hearing Protection
1,NaN,NaN,NaN,NaN,2019-07-02 15:00:00-04:00,Nunavut,NU,2019-06-25,NaN,Aircraft charter to Cambridge Bay from various...
2,NaN,NaN,NaN,NaN,2019-07-02 16:00:00-04:00,"Valleyfield, QC",QC,2019-06-09,NaN,Correctional Clothing and Equipment
3,NaN,NaN,NaN,NaN,2019-07-03 16:00:00-04:00,"Kugluktuk, NU",NU,2019-06-12,NaN,QEC Staff Housing Renovations
4,NaN,NaN,NaN,NaN,2019-07-03 16:00:00-04:00,"Rankin Inlet, NU",NU,2019-06-07,NaN,Replacement of Shop Dust Collector & Duct Work...
5,NaN,NaN,NaN,NaN,2019-07-03 16:00:00-04:00,"Iqaluit, NU",NU,2019-06-14,NaN,Fuel Tank Life-Cycle Replacements
6,NaN,NaN,NaN,NaN,2019-07-04 16:00:00-04:00,"Valleyfield, QC",QC,2019-06-13,NaN,Sealift Office Supplies - Engineering
7,NaN,NaN,NaN,NaN,2019-07-04 14:00:00-04:00,"Ottawa, ON",ON,2019-06-27,NaN,Dental Supplies
8,NaN,NaN,NaN,NaN,2019-07-05 16:00:00-04:00,"Kugluktuk, NU",NU,2019-04-09,NaN,Design-Build New Diesel Power Plant with Solar
9,NaN,NaN,NaN,NaN,2019-07-05 14:00:00-04:00,Nunavut,NU,2019-05-24,NaN,SOA - Nunavut Wildlife Research & Management A...


## Save sample data

In [286]:
sample_data.index = range(len(sample_data))

In [287]:
sample_data.to_json("../sample_data/sample_data.json", orient="records")

In [288]:
sample_data.to_json("../../single-point-of-access-prototype/data/sample_data.json", orient="records")